# Spherical grids and unit converters


In most applications, Parcels works with `spherical` meshes, where longitude and latitude are given in degrees, while depth is given in meters. But it is also possible to use `flat` meshes, where longitude and latitude are given in meters (note that the dimensions are then still called `longitude` and `latitude` for consistency reasons).

In all cases, velocities are given in m/s. Parcels seamlessly converts between meters and degrees, under the hood. For transparency, this guide explains how this works.


Let's first import the relevant modules, and generate a simple dataset on a 2D spherical mesh, with `U`, `V` and `temperature` data arrays, with the velocities 1 m/s and the temperature 20C.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

In [ ]:
from parcels._datasets.structured.generated import simple_UV_dataset

nlat = 10
nlon = 18
ds = simple_UV_dataset(dims=(1, 1, nlat, nlon), mesh="spherical").isel(time=0, depth=0)
ds["temperature"] = ds["U"] + 20  # add temperature field of 20 deg
ds["U"].data[:] = 1.0  # set U to 1 m/s
ds["V"].data[:] = 1.0  # set V to 1 m/s
ds

To create a `parcels.FieldSet` object, we define the `parcels.Field`s and the structured grid (`parcels.XGrid`) the fields are defined on. We add the argument `mesh='spherical'` to the `parcels.XGrid` to signal that all longitudes and latitudes are in degrees.

```{note}
When using a `FieldSet` method for a specific dataset, such as `from_copernicusmarine()`, the grid information is known and parsed by Parcels, so we do not have to add the `mesh` argument.
```

Plotting the `U` field indeed shows a uniform 1 m/s eastward flow.


In [ ]:
grid = parcels.XGrid.from_dataset(ds, mesh="spherical")
U = parcels.Field("U", ds["U"], grid, interp_method=parcels.interpolators.XLinear)
V = parcels.Field("V", ds["V"], grid, interp_method=parcels.interpolators.XLinear)
UV = parcels.VectorField("UV", U, V)
temperature = parcels.Field(
    "temperature", ds["temperature"], grid, interp_method=parcels.interpolators.XLinear
)
fieldset = parcels.FieldSet([U, V, UV, temperature])

plt.pcolormesh(
    fieldset.U.grid.lon,
    fieldset.U.grid.lat,
    fieldset.U.data[0, 0, :, :],
    vmin=0,
    vmax=1,
    shading="gouraud",
)
plt.ylabel("Latitude")
plt.xlabel("Longitude")
plt.colorbar()
plt.show()

However, printing the velocites directly shows something perhaps surprising. Here, we use the square-bracket field-interpolation notation to print the field value at (5W, 40N, 0m depth) at time 0. _Note that sampling a velocity in Parcels is done by calling the `fieldset.UV` VectorField; see the [Field Sampling tutorial](https://docs.oceanparcels.org/en/latest/examples/tutorial_sampling.html#Sampling-velocity-fields) for more information._


In [ ]:
time = np.array([0])
z = np.array([0])
lat = np.array([40])
lon = np.array([-5])
print(fieldset.UV[time, z, lat, lon])
print(
    fieldset.temperature[time, z, lat, lon]
)

While the temperature field indeed is 20C, as we defined, these printed velocities are much smaller.

This is because Parcels converts under the hood from m/s to degrees/s. This conversion is done with a `parcels.converters` object, which is stored in the `.units` attribute of each Field. Below, we print these


In [ ]:
for fld in [fieldset.U, fieldset.V, fieldset.temperature]:
    print(f"{fld.name}: {fld.units}")

So the U field has a `GeographicPolar` UnitConverter object, the V field has a `Geographic` UnitConverter and the `temp` field has a `UnitConverter` object.

Indeed, if we multiply the value of the V field with 1852 \* 60 (the number of meters in 1 degree of latitude), we get the expected 1 m/s.


In [ ]:
u, v = fieldset.UV[time, z, lat, lon]
print(v * 1852 * 60)

Note that you can also interpolate the Field without a unit conversion, by using the `eval()` method and setting `applyConversion=False`, as below


In [ ]:
print(
    fieldset.UV.eval(
        time,
        z,
        lat,
        lon,
        applyConversion=False,
    )
)

## UnitConverters for `mesh='flat'`


If longitudes and latitudes are given in meters, rather than degrees, simply add `mesh='flat'` when creating the XGrid object.


In [ ]:
ds_flat = simple_UV_dataset(dims=(1, 1, nlat, nlon), mesh="flat").isel(time=0, depth=0)
ds_flat["temperature"] = ds_flat["U"] + 20  # add temperature field of 20 deg
ds_flat["U"].data[:] = 1.0  # set U to 1 m/s
ds_flat["V"].data[:] = 1.0  # set V to 1 m/s
grid = parcels.XGrid.from_dataset(ds_flat, mesh="flat")
U = parcels.Field("U", ds_flat["U"], grid, interp_method=parcels.interpolators.XLinear)
V = parcels.Field("V", ds_flat["V"], grid, interp_method=parcels.interpolators.XLinear)
UV = parcels.VectorField("UV", U, V)
temperature = parcels.Field(
    "temperature",
    ds_flat["temperature"],
    grid,
    interp_method=parcels.interpolators.XLinear,
)
fieldset_flat = parcels.FieldSet([U, V, UV, temperature])

plt.pcolormesh(
    fieldset_flat.U.grid.lon,
    fieldset_flat.U.grid.lat,
    fieldset_flat.U.data[0, 0, :, :],
    vmin=0,
    vmax=1,
    shading="gouraud",
)
plt.colorbar()
plt.show()

print(
    "Velocities:",
    fieldset_flat.UV[time, z, lat, lon],
)
for fld in [fieldset_flat.U, fieldset_flat.V, fieldset_flat.temperature]:
    print(f"{fld.name}: {fld.units}")

Indeed, in this case all Fields have the same default `UnitConverter` object.


## UnitConverters for Diffusion fields


The units for Brownian diffusion are in $m^2/s$. If (and only if!) the diffusion fields are called "Kh_zonal" and "Kh_meridional", Parcels will automatically assign the correct Unitconverter objects to these fields.


In [ ]:
kh_zonal = 100  # in m^2/s
kh_meridional = 100  # in m^2/s

ds["Kh_zonal"] = xr.DataArray(
    data=kh_zonal * np.ones((nlat, nlon), dtype=np.float32), dims=["YG", "XG"]
)

kh_zonal_field = parcels.Field(
    "Kh_zonal",
    ds["Kh_zonal"],
    grid=fieldset.U.grid,
    interp_method=parcels.interpolators.XLinear,
)

fieldset.add_field(kh_zonal_field)

ds["Kh_meridional"] = xr.DataArray(
    data=kh_meridional * np.ones((nlat, nlon), dtype=np.float32), dims=["YG", "XG"]
)

kh_meridional_field = parcels.Field(
    "Kh_meridional",
    ds["Kh_meridional"],
    grid=grid,
    interp_method=parcels.interpolators.XLinear,
)

fieldset.add_field(kh_meridional_field)

for fld in [fieldset.Kh_zonal, fieldset.Kh_meridional]:
    val = fld[time, z, lat, lon]
    print(f"{fld.name}: {val} {fld.units}")

Here, the unitconverters are `GeographicPolarSquare` and `GeographicSquare`, respectively.

Indeed, multiplying with $(1852\cdot60)^2$ returns the original value


In [ ]:
deg_to_m = 1852 * 60
print(
    fieldset.Kh_meridional[time, z, lat, lon]
    * deg_to_m**2
)

## Adding a UnitConverter object to a Field


So, to summarise, here is a table with all the conversions

| Field name       | Converter object        | Conversion for `mesh='spherical'`                         | Conversion for `mesh='flat'` |
| ---------------- | ----------------------- | --------------------------------------------------------- | ---------------------------- |
| `"U"`              | `GeographicPolar`       | $1852 \cdot 60 \cdot \cos(lat \cdot \frac{\pi}{180})$     | 1                            |
| `"V"`              | `Geographic`            | $1852 \cdot 60$                                           | 1                            |
| `"Kh_zonal"`       | `GeographicPolarSquare` | $(1852 \cdot 60 \cdot \cos(lat \cdot \frac{\pi}{180}))^2$ | 1                            |
| `"Kh_meridional"`  | `GeographicSquare`      | $(1852 \cdot 60)^2$                                       | 1                            |
| All other fields   | `UnitConverter`         | 1                                                         | 1                            |

Only four Field names are recognised and assigned an automatic UnitConverter object. This means that things might go very wrong when e.g. a velocity field is not called `U` or `V`.

Fortunately, you can always add a UnitConverter later, as explained below:


In [ ]:
ds["Ustokes"] = xr.DataArray(
    data=np.ones((nlat, nlon), dtype=np.float32), dims=["YG", "XG"]
)

fieldset.add_field(
    parcels.Field(
        "Ustokes",
        ds["Ustokes"],
        grid=fieldset.U.grid,
        interp_method=parcels.interpolators.XLinear,
    )
)
print(fieldset.Ustokes[time, z, lat, lon])

This value for `Ustokes` of course is not as expected, since the mesh is spherical and hence this would mean 1 degree/s velocity. Assigning the correct `GeographicPolar` Unitconverter gives


In [ ]:
fieldset.Ustokes.units = parcels.GeographicPolar()
print(fieldset.Ustokes[time, z, lat, lon])
print(
    fieldset.Ustokes[time, z, lat, lon]
    * 1852
    * 60
    * np.cos(40 * np.pi / 180)
)